In [11]:
import pandas as pd
import requests
import numpy as np

In [37]:
URL='https://nycrtspublicportal.azurewebsites.net/data/summary'
response=requests.get(URL)

In [38]:
doc=response.json()

In [39]:
table1=doc[0]
table2=doc[1]
table3=doc[2]
table4=doc[3]

In [40]:
# create a table for reported cases

#create an empty list and dictionary
table1_list=[]
table1_dict={}

table1_dict['date']=table1['title']
table1_dict['total_reported_cases']=table1['rows'][0][0]
table1_dict['total_student_cases']=table1['rows'][0][1]
table1_dict['total_staff_cases']=table1['rows'][0][2]

table1_list.append(table1_dict)
data1=pd.DataFrame(table1_list)

In [41]:
#create an empty list and dictionary
table2_list=[]
table2_dict={}

table2_dict['date']=table2['title']
table2_dict['cum_reported_cases']=table2['rows'][0][0]
table2_dict['cum_student_cases']=table2['rows'][0][1]
table2_dict['cum_staff_cases']=table2['rows'][0][2]

table2_list.append(table2_dict)
data2=pd.DataFrame(table2_list).replace(",","", regex=True)

In [42]:
column_names=[table3['header'][0], table3['header'][1],table3['header'][2],table3['header'][3]]
data3=pd.DataFrame(table3['rows'])
data3.columns=column_names
trans_data3=data3.set_index('Action').T.reset_index().rename(columns={
    'index':'doe_dece_schools'}).replace(",","", regex=True)
trans_data3['date']=trans_data3.doe_dece_schools.str.split('as', expand=True)[0][0].replace('<br>',"")

In [43]:
column_names=[table4['header'][0], table4['header'][1],table4['header'][2],table4['header'][3]]
data4=pd.DataFrame(table4['rows'])
data4.columns=column_names
trans_data4=data4.set_index('Action').T.reset_index().rename(columns={
    'index':'charter_action'}).replace(",","", regex=True)
trans_data4['date']=trans_data4.charter_action.str.split('as', expand=True)[0][0].replace('<br>',"")


In [44]:
old_data1=pd.read_csv("data/reported_covid_cases.csv")
old_data2=pd.read_csv("data/cum_covid_cases.csv")
old_data3=pd.read_csv("data/doe_dece_actions.csv")
old_data4=pd.read_csv("data/charter_actions.csv")

In [45]:
merged_data1=pd.concat([data1, old_data1]).reset_index(drop=True).drop_duplicates(subset=['date'])
merged_data2=pd.concat([data2, old_data2]).reset_index(drop=True).drop_duplicates(subset=['date'])
merged_data3=pd.concat([trans_data3, old_data3]).reset_index(drop=True).drop_duplicates(subset=['date'])
merged_data4=pd.concat([trans_data4, old_data4]).reset_index(drop=True).drop_duplicates(subset=['date'])

In [46]:
merged_data1.to_csv("data/reported_covid_cases.csv", index=False)
merged_data2.to_csv("data/cum_covid_cases.csv", index=False)
merged_data3.to_csv('data/doe_dece_actions.csv', index=False)
merged_data4.to_csv('data/charter_actions.csv', index=False)